# to generate illumina nex and ht sample sheet with barcodes

In [60]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
import os,pickle
import csv
import sys
import glob
from collections import OrderedDict,defaultdict,Counter
import datetime
%matplotlib inline 

import xlsxwriter
from scipy import stats

import seaborn as sns
# from pandas.tools import plotting
from matplotlib import pyplot as plt


from Bio.Alphabet import generic_dna,generic_rna,generic_protein
import primer3
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import math
import pysam



In [26]:
def primer_loc(rows,cols):
    locs=list()
    for r5 in rows:
        for c5 in cols:
             locs.append(str(r5)+str(c5))
    return locs

In [27]:
### combine i5, i7 by rows: A:A; B:B....H:H
def xbarcodes_pairs(rows_i5='ABCD',rows_i7='ABCD',cols_i5='5678',cols_i7='5678'):
    barcodes_i5=primer_loc(rows_i5,cols_i5)
    barcodes_i7=primer_loc(rows_i7,cols_i7)
    xbarcodes_i5=list()
    for j in barcodes_i5:
        for i in range(len(cols_i7)):
            xbarcodes_i5.append(j)
    barcodes_pair=list(zip(xbarcodes_i5,barcodes_i7))
    return barcodes_pair
#     return xbarcodes_i5

In [2]:
current_folder_path, current_folder_name = os.path.split(os.getcwd())


In [3]:
# define the name of the directory to be created
samples_folder = current_folder_path+'/samples'

try:  
    os.mkdir(samples_folder)
except OSError:  
    print ("Creation of the directory %s failed" % samples_folder)
else:  
    print ("Successfully created the directory %s " % samples_folder)

Successfully created the directory /Users/ltao/Projects/glioma/exp/190501/samples 


In [5]:
# try:  
#     os.rmdir(path)
# except OSError:  
#     print ("Deletion of the directory %s failed" % path)
# else:  
#     print ("Successfully deleted the directory %s" % path)

In [61]:
pickle_name='sample_sheet.pickle'

# out_pickle=pickle_path+pickle_name
pickle_out = open(pickle_name,"wb")
pickle.dump(sample_sheet, pickle_out)
pickle_out.close()
# # load
# pickle_in = open(out_pickle,"rb")
# scibc = pickle.load(pickle_in)


In [58]:
## Setting Sample Sheet
exp_name='glioma_sc_mda_mpcr'
exp_date= str(datetime.datetime.today().date())
read1_len=250
read2_len=250
# read1_len=150
# read2_len=150

In [56]:
# samplesheet_file='/Users/ltao/Projects/Data/miseq/nex/MS8001943-300V2.csv'
# sample_sheet = pd.read_csv(samplesheet_file)
# input_key = sample_sheet.iloc[[19]].values.tolist()[0]
# sample_sheet.columns=input_key
# sample_sheet=sample_sheet.drop([20])
# sample_sheet.loc[2,'Sample_Name']=exp_name
# sample_sheet.loc[3,'Sample_Name']=exp_date
# sample_sheet.loc[11,'Sample_ID']=read1_len
# sample_sheet.loc[12,'Sample_ID']=read2_len

# sample_sheet

In [66]:
sample_sheet_pickle = open("sample_sheet.pickle","rb")

sample_sheet = pickle.load(sample_sheet_pickle)
sample_sheet.loc[2,'Sample_Name']=exp_name
sample_sheet.loc[3,'Sample_Name']=exp_date
sample_sheet.loc[11,'Sample_ID']=read1_len
sample_sheet.loc[12,'Sample_ID']=read2_len

In [32]:
# ## get nexetra adaptors
# sheets = pd.read_excel('/Users/ltao/Projects/Data/miseq/nex/Nextra-Ads.xlsx', sheet_name=None)
# nexads=dict()
# # for sheet_name, sheet in sheets.items():
# #     print (sheet_name)
# nex_i5=sheets["plate AD1-i5"]
# nex_i5.set_index('96 Well Columns', inplace=True)
# nex_i7=sheets["plate Ad2 i7"]
# nex_i7.set_index('96 Well Columns', inplace=True)
# layout_i5=sheets['layout_i5']
# layout_i7=sheets['layout_i7']
# nex_i5_dic=nex_i5.to_dict(orient="index")
# nex_i7_dic=nex_i7.to_dict(orient="index")

In [71]:

nex_i5_dic_pickle = open("nex_i5_dic.pickle","rb")
nex_i5_dic = pickle.load(nex_i5_dic_pickle)

nex_i7_dic_pickle = open("nex_i7_dic.pickle","rb")
nex_i7_dic = pickle.load(nex_i7_dic_pickle)

In [69]:
pickle_name='nex_i7_dic.pickle'

# out_pickle=pickle_path+pickle_name
pickle_out = open(pickle_name,"wb")
pickle.dump(nex_i7_dic, pickle_out)
pickle_out.close()
# # load
# pickle_in = open(out_pickle,"rb")
# scibc = pickle.load(pickle_in)


In [28]:

# non-dig-hcc38-0.7xbeads:

# Ad1: col 11
# Ad2:1-6


rows_i5='ABCDEFGH'
rows_i7='ABCDEFGH'
cols_i5=[4,]
cols_i7=[1,2]

idh=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[4,]
cols_i7=[3,4]
# cols_i7=[7,8,9,10,11,12]

g_5x=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[4,]
cols_i7=[5,6]

g_17x_a=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[5,]
cols_i7=[7,]

g_17x_b=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)
g_17x =g_17x_b+g_17x_a

cols_i5=[5,]
cols_i7=[8]
hcc38_g_17x_q5=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[5,]
cols_i7=[9,10]

g_17x_q5=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)



In [34]:
### chose runtype
color = int(input("NovaSeq, MiSeq, HiSeq 2000/2500 == four_color, \n MiniSeq, NextSeq, HiSeq 3000/4000 == two_color, \n what is your run? \n pls input 2 or 4\n"))

if color == 4:
    runtype ="four_color"
    print ("your run is ###four_color###")
elif color == 2:
    runtype ="two_color"
    print ("your run is ###two_color###")
else:
    print ('pls define your runtype, 2 or 4')

NovaSeq, MiSeq, HiSeq 2000/2500 == four_color, 
 MiniSeq, NextSeq, HiSeq 3000/4000 == two_color, 
 what is your run? 
 pls input 2 or 4
4
your run is ###four_color###


In [42]:
input_bp=OrderedDict()

## for NovaSeq, MiSeq, HiSeq 2000/2500, 
for plate,bps in {"idh":idh,"g_5x":g_5x,"g_17x":g_17x,"hcc38_pbs":hcc38_g_17x_q5,"g_17x_q5":g_17x_q5}.items():
    for bp in bps:
        samples_input_dict=OrderedDict()
        if runtype == "four_color":
            i5_barcodes = nex_i5_dic[bp[0]]['i5 Bases for Sample Sheet NovaSeq, MiSeq, HiSeq 2000/2500\xa0']
        else:
            i5_barcodes = nex_i5_dic[bp[0]]['i5 Bases for Sample Sheet MiniSeq, NextSeq, HiSeq 3000/4000 ']

        i7_barcodes = nex_i7_dic[bp[1]]['on Sample Sheet']
        for k in input_key:
            samples_input_dict[k] = 'i7_'+bp[1]+'_i5_'+bp[0]
        samples_input_dict['index2'] =i5_barcodes
        samples_input_dict['index'] =i7_barcodes
        samples_input_dict['Sample_Plate'] = plate
        input_bp['i7_'+bp[1]+'_i5_'+bp[0]]=samples_input_dict


In [ ]:
# barcodes_file=samples_folder+'/barcodes.xlsx'

# input_bp_df=pd.DataFrame.from_dict(input_bp, orient='index')
# input_bp_df.to_excel(barcodes_file)

In [45]:
# runsample_sheet=samples_folder+'/runsample_sheet.csv'

input_bp_df=pd.DataFrame.from_dict(input_bp, orient='index')
result = sample_sheet.append(input_bp_df)
# result.to_csv(runsample_sheet,header=False, index=False)
result.to_csv("runsample_sheet.csv",header=False, index=False)

### Prepare all fastqs by shell in merge_script


In [17]:
fastqs_folder=current_folder_path+'/fastqs/'
os.mkdir(fastqs_folder)

In [17]:
# fastqs_folder=current_folder_path+'/fastqs/'
# os.mkdir(fastqs_folder)
merge_next_file=fastqs_folder+"merge_nex.txt"
# prepare merge config files
fastqs=list()
for x in os.listdir(fastqs_folder):
    if 'fastq' in x:
        fastqs.append(x)
fastqs=sorted(fastqs
             )
merge_key=['sample','r1','r2']
merge_nex=OrderedDict()

for x in zip(fastqs[::2],fastqs[1::2]):
    pe_reads=dict()
    sindex=x[0].index('_S')
    name=x[0][:sindex]
#     print (name)
    mk=input_bp[name]["Sample_Plate"]+"_"+name
    pe_reads['sample']=mk
    pe_reads ['r1']=x[0]
    pe_reads['r2']=x[1]
    merge_nex[mk]=pe_reads
df_merge_nex=pd.DataFrame.from_dict(merge_nex, orient='index')
df_merge_nex.to_csv(merge_next_file,sep="\t")  
# merge_nex

In [4]:
fastqs_folder=current_folder_path+'/fastqs/'
